##This notebook is created to check the shapes and output of each layers in the model

In [1]:
import sys
sys.path.insert(0, "..")

## DATA LOADING FUNCTION WHICH RETURN ONLY 10 Data for Training and 10 Data for testing

In [4]:
import os
import random
from os import listdir
from os.path import isfile, isdir, join
from collections import defaultdict as ddict
import torch
from sklearn.model_selection import train_test_split
from PIL import Image


def extract_data_path_concept_and_label(data_dir,
                                    test_ratio = 0.2,
                                    stratify = True):
    cwd = os.getcwd()
    data_path = join(cwd, data_dir + '/images')
    val_ratio = 0.2

    path_to_id_map = dict() #-----map from full image path to image id
    with open(data_path.replace('images', 'images.txt'), 'r') as f:
        for line in f:
            items = line.strip().split()
            key_str = join(data_path, items[1]).replace('\\', '/')
            path_to_id_map[key_str] = int(items[0])

    attribute_labels_all = ddict(list)      #---------------------map from image id to a list of attribute labels
    attribute_certainties_all = ddict(list)   #-------------------map from image id to a list of attribute certainties
    attribute_uncertain_labels_all = ddict(list) #----------------map from image id to a list of attribute labels calibrated for uncertainty
    # 1 = not visible, 2 = guessing, 3 = probably, 4 = definitely
    uncertainty_map = {1: {1: 0, 2: 0.5, 3: 0.75, 4:1}, #calibrate main label based on uncertainty label
                        0: {1: 0, 2: 0.5, 3: 0.25, 4: 0}}
    with open(join(cwd, data_dir + '/attributes/image_attribute_labels.txt'), 'r') as f:
        for line in f:
            file_idx, attribute_idx, attribute_label, attribute_certainty = line.strip().split()[:4]
            attribute_label = int(attribute_label)
            attribute_certainty = int(attribute_certainty)
            uncertain_label = uncertainty_map[attribute_label][attribute_certainty]
            attribute_labels_all[int(file_idx)].append(attribute_label)
            attribute_uncertain_labels_all[int(file_idx)].append(uncertain_label)
            attribute_certainties_all[int(file_idx)].append(attribute_certainty)

    is_train_test = dict() #map from image id to 0 / 1 (1 = train)
    with open(join(cwd, data_dir + '/train_test_split.txt'), 'r') as f:
        for line in f:
            idx, is_train = line.strip().split()
            is_train_test[int(idx)] = int(is_train)
    print("Number of train images from official train test split:", sum(list(is_train_test.values())))

    X_metadata=[]
    y_labels =[]
    folder_list = [f for f in listdir(data_path) if isdir(join(data_path, f))]
    folder_list.sort() #sort by class index
    for i, folder in enumerate(folder_list):
        folder_path = join(data_path, folder)
        classfile_list = [cf for cf in listdir(folder_path) if (isfile(join(folder_path,cf)) and cf[0] != '.')]
        #classfile_list.sort()
        for cf in classfile_list:
            key_str = join(folder_path, cf).replace('\\', '/')
            img_id = path_to_id_map[key_str]
            img_path = join(folder_path, cf).replace('\\', '/')
            img = Image.open(img_path)
            if img.mode != 'RGB':  #------- since there are a few gray scale images, we just skipped them
                continue
            metadata = {'id': img_id, 'img_path': img_path,
                      'attribute_label': attribute_labels_all[img_id] }
            X_metadata.append(metadata)
            y_labels.append(i)
    X_train, X_test, y_train, y_test = train_test_split(X_metadata, y_labels, train_size=0.8, random_state=42, stratify=y_labels)
    return X_train, X_test, y_train, y_test 

In [5]:
data_dir ="../dataset/CUB_200_2011/"
X_train, X_test, y_train, y_test = extract_data_path_concept_and_label(data_dir)
print(f"Train data: {len(X_train)} and Test Data: {len(X_test)}")

Number of train images from official train test split: 5994
Train data: 9424 and Test Data: 2356


In [10]:
type(X_test[0]['attribute_label'])

int

In [11]:
import pickle
with open('../dataset/cub_pickles/X_train.pkl', 'wb') as f:
    pickle.dump(X_train, f)
f.close()

In [12]:
import pickle
with open('../dataset/cub_pickles/X_test.pkl', 'wb') as f:
    pickle.dump(X_test, f)
f.close()

In [13]:
import pickle
with open('../dataset/cub_pickles/y_train.pkl', 'wb') as f:
    pickle.dump(y_train, f)
f.close()

In [14]:
import pickle
with open('../dataset/cub_pickles/y_test.pkl', 'wb') as f:
    pickle.dump(y_test, f)
f.close()

In [15]:
with open('../dataset/cub_pickles/X_test.pkl', 'rb') as f:
    test2 = pickle.load(f)
f.close()

In [20]:
type(test2[0]['attribute_label'][0])

int

In [7]:
from source import models
import torch
from torch import nn

device = "cuda" if torch.cuda.is_available() else "cpu"

model = models.CBM(c_chanel=1, 
                   hidden_units=32, 
                   concepts=312, 
                   output_shape=200,
                   img_shape=28).to(device)

#TRAIN THE MODEL AND INSPECT

In [14]:
import torch
from torch import nn
from torchvision import transforms
from PIL import Image
import numpy
import matplotlib.pyplot as plt  
from source import data_processing


epochs = 1
BATCH_SIZE = 5
img_shape = 28
learning_rate = 0.01
consider_concept_loss = True


#loss_fn_concept = nn.CrossEntropyLoss()
loss_fn_concept = nn.BCEWithLogitsLoss()  #-------- loss function for concept identification 
loss_fn_target = nn.CrossEntropyLoss()    #-------- loss function for class label identifaction


optimizer = torch.optim.SGD(params=model.parameters(), lr=learning_rate) #------ optimizer SGD (Stochhastic Gradient Descent)

total_train_data =len(train_data)
total_batch = (int) (total_train_data/BATCH_SIZE)

resize = transforms.Resize( size=(img_shape,img_shape) ) #---- resize the image 
gray = transforms.Grayscale() #----- convert into gray scale 
convert_tensor = transforms.ToTensor() #------- transform the image into tensor

loss_values=[]
concept_loss = []
target_loss = []

#---------------TRAINING LOOP ------------###
flag=True
for epoch in range(epochs):
    train_loss=0.0
    concept_loss_per_epoch =0.0
    target_loss_per_epoch = 0.0
    #print(f"------Epoch {epoch+1} is in progress--------")

    for batch in range(total_batch):
        X_list =[]
        concept_list =[]
        class_list =[]

        for i in range(BATCH_SIZE):    #------LOOP to load the data for each batch -------
            img = Image.open(train_data[ batch*BATCH_SIZE+i ]['img_path'])
            # if img.mode != 'RGB':  #------ To find out the gray scale images in CUB-----
            #     print(f"Image Path: {train_data[ batch*BATCH_SIZE+i ]['img_path']}")
            img =convert_tensor(gray(resize(img)))
            img_np = img.numpy()
            X_list.append(img_np)  #-----appending the image-----
            concept_list.append(train_data[ batch*BATCH_SIZE+i]['attribute_label']) #----appending the concepts------
            class_list.append(train_data[ batch*BATCH_SIZE+i]['class_label']) #----appending the class label -----

        #------- move dataset to GPU -------
        X = torch.tensor(X_list).to(device)
        concepts = torch.tensor(concept_list).to(device)
        y = torch.tensor(class_list).to(device)
        
        model.train()  #----- putting model into training mode ------
        y_logits, c_hats = model(X)

        optimizer.zero_grad()

        loss1 = loss_fn_concept(c_hats, concepts.float())
        loss2 = loss_fn_target(y_logits, y)
        if flag: 
            print(f"Shape of y_logits: {y_logits.shape},\n Shape of c_hat: {c_hats.shape}\n Y Shape: {y.shape}\n Concept Shape: {concepts.shape}\n ")
            print(f"Y LOGITS:\n {y_logits}\n")
            print(f"Y:\n {y}\n")
            print(f"Y (argmax): {len(y_logits.argmax(dim=0))} \n{y_logits.argmax(dim=1)}")
            print(f"C_HAT:\n {c_hats}\n")
            print(f"Concepts:\n {concepts}")
            print(f"Loss 1: {loss1}")
            print(f"Loss 2: {loss2}")

        if consider_concept_loss:
            loss = loss1+loss2
        else:
            loss = loss2
        concept_loss_per_epoch +=loss1
        target_loss_per_epoch += loss2
        train_loss += loss
        loss.backward()

        optimizer.step()
        # if batch%50==0:
        #     print(f"Total {(batch+1)*BATCH_SIZE} of {total_batch*BATCH_SIZE} data processed")
    train_loss /= (total_batch*BATCH_SIZE)
    concept_loss_per_epoch /= (total_batch*BATCH_SIZE)
    target_loss_per_epoch /= (total_batch*BATCH_SIZE)
    loss_values.append(train_loss)
    concept_loss.append(concept_loss_per_epoch)
    target_loss.append(target_loss_per_epoch)
    print(f"Epoch {epoch+1}: Training loss {train_loss: .5f}, concept loss: {concept_loss_per_epoch: .5f}, target loss: {target_loss_per_epoch: .5f}\n")

Shape of y_logits: torch.Size([5, 200]),
 Shape of c_hat: torch.Size([5, 312])
 Y Shape: torch.Size([5])
 Concept Shape: torch.Size([5, 312])
 
Y LOGITS:
 tensor([[-0.0428,  0.0793, -0.0303, -0.0035, -0.0021, -0.0254, -0.0218, -0.0618,
          0.0190, -0.0428, -0.0858,  0.0348,  0.0159, -0.0379,  0.0278, -0.0334,
          0.0458, -0.0507, -0.0240,  0.0253, -0.0945,  0.0202, -0.0402, -0.0008,
          0.0439, -0.0365,  0.0167,  0.0118, -0.0426, -0.0183, -0.1121,  0.0092,
         -0.0957, -0.0626, -0.0469,  0.0244, -0.0080, -0.0590, -0.0817, -0.0784,
          0.0122,  0.0238,  0.0773, -0.0077, -0.0199,  0.0722,  0.0397, -0.0165,
         -0.0065, -0.0543,  0.0119,  0.0309,  0.0538, -0.0545, -0.1195,  0.0311,
          0.0029, -0.0314,  0.0887, -0.0063, -0.0412,  0.1043,  0.0497,  0.0510,
          0.0143,  0.0433, -0.0002, -0.0246,  0.0378,  0.0266,  0.0092, -0.0318,
         -0.0844, -0.0158,  0.0196,  0.0455, -0.0366,  0.0307,  0.0660, -0.0677,
         -0.0804,  0.0119, -0.0888,

In [16]:
tensor1 = torch.arange(0.5, 3.0, 0.35)
tensor1

tensor([0.5000, 0.8500, 1.2000, 1.5500, 1.9000, 2.2500, 2.6000, 2.9500])

In [22]:
out = nn.functional.sigmoid(tensor1)
print(out)
print(sum(out))

tensor([0.6225, 0.7006, 0.7685, 0.8249, 0.8699, 0.9047, 0.9309, 0.9503])
tensor(6.5721)


In [20]:
out = nn.functional.softmax(tensor1)
out

C:\Users\DELL\AppData\Local\Temp\ipykernel_17004\416900055.py:1: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  out = nn.functional.softmax(tensor1)


tensor([0.0271, 0.0385, 0.0546, 0.0775, 0.1100, 0.1561, 0.2216, 0.3144])

In [23]:
out.argmax()

tensor(7)

##### LOAD THE TRAINED MODEL TO CHECK THE CF MATRIX 

In [1]:
import sys
sys.path.insert(0, "..")

import torch
from source.test_model import test_with_CUB 
trained_model = torch.load("../saved_models/model_100_iteration.pickle")

In [2]:
from source import data_processing, train_model

dir ="../dataset/CUB_200_2011/"   #-----directory of the CUB dataset
#train_data, val_data, test_data = data_processing.extract_data(dir)  
train_data, test_data = data_processing.extract_data(dir)  #-----data_processing function is modified not to split the validation set for now
print(f"Train data: {len(train_data)} and Test Data: {len(test_data)}")

device = "cpu"

c:\Users\DELL\anaconda3\envs\py3.10\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Number of train images from official train test split: 5994
Train data: 5994 and Test Data: 5794


In [4]:
test_loss, concept_loss, target_loss, test_acc, precisions, recalls, CF = test_with_CUB(trained_model,
              test_data=test_data,
              device=device,
              n_concepts=213,
              n_labels=200
              )

TEST LOSS: 0.00198 CONCEPT LOSS:  0.00015 TARGET LOSS:  0.00183

TARGET ACCURACY:  0.02209

Precisions:
[0.03333333333333333, 0.0, 0.0, 0.0, 0.07142857142857142, 0.0, 0.043478260869565216, 0.0, 0.06896551724137931, 0.0, 0.0, 0.07692307692307693, 0.13333333333333333, 0.0, 0.03571428571428571, 0.0, 0.037037037037037035, 0.0, 0.0, 0.0, 0.0, 0.0, 0.034482758620689655, 0.045454545454545456, 0.03333333333333333, 0.2, 0.0, 0.0, 0.1, 0.0, 0.03333333333333333, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.03333333333333333, 0.0, 0.0, 0.0, 0.03333333333333333, 0.0, 0.06666666666666667, 0.03333333333333333, 0.03333333333333333, 0.03333333333333333, 0.03333333333333333, 0.06666666666666667, 0.03333333333333333, 0.1, 0.03333333333333333, 0.03333333333333333, 0.03333333333333333, 0.0, 0.0, 0.0, 0.034482758620689655, 0.1, 0.0, 0.06666666666666667, 0.0, 0.0, 0.0, 0.0, 0.03333333333333333, 0.0, 0.03333333333333333, 0.0, 0.0, 0.0, 0.0, 0.037037037037037035, 0.03333333333333333, 0.03333333333333333, 0.034482

In [1]:
import torch 
my_tensor1 = torch.rand(3,4)
my_tensor2 = torch.tensor([[1,0,0],
                           [0,1,0]])

In [2]:
my_tensor1

tensor([[0.6552, 0.8726, 0.4600, 0.3294],
        [0.1064, 0.7575, 0.4658, 0.9441],
        [0.1933, 0.0210, 0.5533, 0.5822]])

In [3]:
my_tensor2

tensor([[1, 0, 0],
        [0, 1, 0]])

In [12]:
indices = torch.arange(1,3,1)
tensor3 = torch.index_select(my_tensor1, 1, indices)
tensor3

tensor([[0.8726, 0.4600],
        [0.7575, 0.4658],
        [0.0210, 0.5533]])

In [15]:
tensor3.argmax(1)

tensor([0, 0, 1])

In [1]:
import sys
sys.path.insert(0, "..")

In [2]:
import pickle 
with open('../dataset/cub_pickles/X_test.pkl', 'rb') as f:
    X_test = pickle.load(f)
f.close()
import pickle 
with open('../dataset/cub_pickles/y_test.pkl', 'rb') as f:
    y_test = pickle.load(f)
f.close()

In [3]:
model = pickle.load(open('../saved_models/model_1_1.pkl', 'rb'))

In [4]:
from source.test_model import test_with_CUB
test_with_CUB(model=model,
              X_test=X_test,
              y_test=y_test,
              device='cpu',
              )

d:\ML_Projects\CBM\jupyter_notebooks\..\source\test_model.py:49: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\cb\pytorch_1000000000000\work\torch\csrc\utils\tensor_new.cpp:248.)
  X_test = torch.tensor(X_list).to(device)


TEST LOSS: 0.00000 CONCEPT LOSS:  0.00000 TARGET LOSS:  0.00000

TARGET ACCURACY:  0.07895

Precisions:
[0.09090909 0.         0.         0.21428571 0.         0.
 0.         0.         0.         0.07692308 0.2        0.14285714
 0.15384615 0.25       0.11764706 0.27272727 0.1875     0.16666667
 0.18181818 0.33333333 0.         0.         0.         0.
 0.18181818 0.23076923 0.         0.         0.11111111 0.
 0.         0.13333333 0.11111111 0.         0.1        0.
 0.07692308 0.         0.         0.05882353 0.         0.5
 0.         0.14285714 0.2        0.08333333 0.06666667 0.
 0.05882353 0.07142857 0.125      0.2        0.         0.16
 0.1        0.         0.25       0.         0.08333333 0.125
 0.25       0.11111111 0.         0.         0.         0.33333333
 0.125      0.14285714 0.2        0.4        0.06666667 0.08333333
 0.         0.28571429 0.         0.         0.14285714 0.
 0.0625     0.         0.         0.         0.25       0.
 0.25       0.15384615 0.2352941

c:\Users\DELL\anaconda3\envs\py3.10\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\DELL\anaconda3\envs\py3.10\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\DELL\anaconda3\envs\py3.10\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\DELL\anaconda

In [6]:
file1 = open("../output_files/output_model_1_1.txt", "a")
x=30
file1.write(f"value of x is {x}\n")
file1.close()